In [2]:
# --- Librerías ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, interactive, Layout

# --- Cargar archivo CSV ---
RUTA = '/workspaces/ml-bootcamp-labs/data/raw/Shift_Pozos_LDiscEoc-cvs.csv'
df = pd.read_csv(RUTA, sep=';')

# --- Normalizar nombres de columnas ---
df.columns = df.columns.str.strip().str.lower()

# Verifica nombres de columnas disponibles
print(df.columns.tolist())

# --- Función de estadísticos personalizados ---
def stats(x):
    return pd.Series({
        'count': x.count(),
        'mean': x.mean(),
        'std': x.std(),
        'min': x.min(),
        'p10': np.percentile(x, 10),
        'p50': np.percentile(x, 50),
        'p90': np.percentile(x, 90),
        'max': x.max()
    })

# --- Estadísticos globales ---
print("\nEstadísticos generales de 'delta':")
print(df['delta'].describe(percentiles=[0.1, 0.5, 0.9]))

# --- Función principal de análisis ---
def analisis(grupo_por, pozos):
    if not pozos:
        print("Selecciona al menos un pozo")
        return
    
    # Filtrar solo los pozos seleccionados
    df_filtrado = df[df[grupo_por].isin(pozos)]
    
    # Estadísticos por pozo
    stats_group = df_filtrado.groupby(grupo_por)['delta'].apply(stats).reset_index()
    display(stats_group)
    
    # Histograma comparativo
    plt.figure(figsize=(10,6))
    for pozo in pozos:
        sns.kdeplot(df_filtrado[df_filtrado[grupo_por]==pozo]['delta'], label=str(pozo), fill=True, alpha=0.3)
    plt.title(f"Distribución de Delta - {grupo_por}")
    plt.legend()
    plt.show()
    
    # Boxplot comparativo
    plt.figure(figsize=(12,6))
    sns.boxplot(x=grupo_por, y='delta', data=df_filtrado)
    plt.title(f"Boxplot de Delta - {grupo_por}")
    plt.show()
    
    # Mapa espacial (todos los pozos seleccionados)
    plt.figure(figsize=(8,6))
    sc = plt.scatter(df_filtrado['x'], df_filtrado['y'], c=df_filtrado['delta'], cmap='coolwarm', s=80, edgecolor='k')
    plt.colorbar(sc, label="Delta")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title(f"Distribución espacial de Delta - {grupo_por}")
    plt.show()

# --- Widgets ---
grupo_selector = widgets.Dropdown(
    options=['uwi', 'common well name'],
    value='uwi',
    description='Agrupar por:'
)

pozo_selector = widgets.SelectMultiple(
    options=df['uwi'].dropna().unique(),
    description='Pozos:',
    layout=Layout(width='50%')
)

# Actualizar opciones de pozos según grupo seleccionado
def update_pozo_options(*args):
    pozo_selector.options = df[grupo_selector.value].dropna().unique()

grupo_selector.observe(update_pozo_options, 'value')

# --- Interacción ---
interactive_plot = interactive(analisis, grupo_por=grupo_selector, pozos=pozo_selector)
display(interactive_plot)


['well id', 'uwi', 'common well name', 'x', 'y', 'pick name', 'pick z', 'nearest node z', 'surface intersection z', 'delta', 'delta reverse', 'absolute delta', 'interpreter id', 'observation #']

Estadísticos generales de 'delta':
count    122.000000
mean       6.749800
std      146.858049
min     -328.943573
10%     -178.887060
50%        6.945914
90%      166.233730
max      791.108127
Name: delta, dtype: float64


interactive(children=(Dropdown(description='Agrupar por:', options=('uwi', 'common well name'), value='uwi'), …